<a href="https://colab.research.google.com/github/ashishpatidar123/ashishpatidar123/blob/main/Model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#import all the required libraries
import tensorflow as tf
from keras.layers import Conv3D,BatchNormalization,ReLU,Flatten,Reshape,MaxPool3D, add ,Dense
from keras.models import Model
from math import floor
import numpy as np

#Now we store the dimensions of the input.

Nr=1
Nt=8
K=52
# define the network
def network(x):

    def ConvBlock1(y):
        y = Conv3D(4*L, kernel_size = (3,7,5),padding = 'same', strides =(1,1,1))(y)
        y = BatchNormalization()(y)
        y = ReLU()(y)
        return y
    
    def Maxpool1(y):
        y = MaxPool3D(pool_size = (3,3,3), padding = 'same', strides = (1,1,1))(y)
        return y
    
    def ConvResBlock(y):
        
        temp = y
        y = Conv3D(8*L, kernel_size = (3,7,5), padding = 'same', strides = (1,1,1))(y)
        y = Conv3D(16*L, kernel_size = (3,7,5), padding = 'same', strides = (1,1,1))(y)
        y = Conv3D(4*L, kernel_size = (3,7,5), padding = 'same', strides = (1,1,1))(y)
        y = add([temp,y])

        y = BatchNormalization()(y)
        y = ReLU()(y)

        return y
    
    def ConvBlock2(y):
        y = Conv3D(2, kernel_size = (3,7,7),padding = 'same', strides =(1,1,1))(y)
        y = BatchNormalization()(y)
        y = ReLU()(y)
        return y
    
    def Maxpool2(y):
        y = MaxPool3D(pool_size = (4,1,2), padding ='valid', strides = (4,1,2))(y)
        return y
    
    def FCBlock(y):
        #y = y.Reshape((B,2,math.floor(Nr/1),math.floor(Nr/2),math.floor(Nr/4)))(y)
        y = Flatten()(x)
        y = Dense(2*Nr*Nt*K, activation = 'linear')(y)
        y = Reshape((2,Nr,Nt,K))(y)
        return y
    
    
    x = ConvBlock1(x)
    x = Maxpool1(x)
    temp2 = x
    x = ConvResBlock(x)
    x = ConvResBlock(x)
    x = add([temp2,x])

    x = ConvBlock2(x)
    x = Maxpool2(x)
    x = FCBlock(x)

    return x
  
  
        
    
    
    
    
import numpy as np
import tensorflow as tf
import scipy.io as sio
from keras.layers import Input
from keras.models import Model
from keras.callbacks import TensorBoard, Callback
import matplotlib.pyplot as py
import numpy as np
# import mat73
import time
import math

# from NeuralNetwork import Neural_Network
batch_sizer = 512
Nr = 1
Nt = 8
K = 52

!pip install h5py
import h5py




L = 3


# (52, 8, 1, 8, 85779)
# (52, 8, 1, 8, 22222)

# print(input_train_samples.shape)
# print(input_test_samples.shape)
# print(output_train_samples.shape)
# print(output_test_samples.shape)

# (85779, 6, 1, 8, 52)
# (22222, 6, 1, 8, 52)
# (85779, 2, 1, 8, 52)
# (22222, 2, 1, 8, 52)
# I'm getting ram error so I created the modified files in advance
input_train_samples = np.load('/content/drive/MyDrive/Datasets/input_train_samples_modified.npy')
output_train_samples = np.load('/content/drive/MyDrive/Datasets/output_train_samples_modified.npy')

# defining the input tensor
input_tensor = Input(shape=(2*L, Nr, Nt, K))
output_value = network(input_tensor)

initial_lr = 1e-3
# defining the optimiser, loss function for out model
CsiNet = Model(inputs=[input_tensor], outputs=[output_value])
CsiNet.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=initial_lr), loss='mse')
print(CsiNet.summary())

lr_drop_period1 = 100
lr_drop_period2 = 200
lr_drop_period3 = 250
lr_drop_factor = 0.1

# this piece of code will help in storing the losses
class LossHistory(Callback):
    # At the start of our training we intialize 2 variables to keep track of our losses
    def on_train_begin(self, logs={}):
        self.losses_train = []
        self.losses_val = []
    # We append to the end of the losses_train our current batch loss

    def on_batch_end(self, batch, logs={}):
        self.losses_train.append(logs.get('loss'))
    # We append to the end of the losses_val our validation loss

    def on_epoch_end(self, epoch, logs={}):
        self.losses_val.append(logs.get('val_loss'))

    def scheduler(epoch, lr):
        if lr_drop_period1 == np.Inf or np.mod(epoch, lr_drop_period1) != 0 or lr_drop_period2 == np.Inf or np.mod(epoch, lr_drop_period2) != 0 or lr_drop_period3 == np.Inf or np.mod(epoch, lr_drop_period3) != 0:
            return lr
        else:
            return lr * tf.math.exp(-lr_drop_factor)


history = LossHistory()
# Create a file name
file = 'CsiNet_'+'_dim'+time.strftime('_%m_%d')
path = 'result/TensorBoard_%s' % file
earlyStopping = tf.keras.callbacks.EarlyStopping(
    patience=200, restore_best_weights=True)
#  We now train our model with our data with number of steps=1000, (samples per size =200) and shuffle data with each step
#  Also validate using x_val as both input and target
#  We also need to record the losses using callbacks




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 6, 1, 8, 52  0           []                               
                                )]                                                                
                                                                                                  
 conv3d (Conv3D)                (None, 6, 1, 8, 12)  65532       ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 6, 1, 8, 12)  48         ['conv3d[0][0]']                 
 alization)                                                                                   

In [4]:
CsiNet.fit(input_train_samples, output_train_samples,
           epochs=300,
           batch_size=512,
           validation_data=(input_train_samples, output_train_samples),
           shuffle=True,
           callbacks=[history,
                      TensorBoard(log_dir=path), earlyStopping])

Epoch 1/300
  6/168 [>.............................] - ETA: 11s - loss: 0.0171

168/168 [==============================] - 34s 107ms/step - loss: 0.0050 - val_loss: 0.0046
Epoch 2/300
168/168 [==============================] - 15s 89ms/step - loss: 0.0044 - val_loss: 0.0045
Epoch 3/300
168/168 [==============================] - 15s 90ms/step - loss: 0.0044 - val_loss: 0.0044
Epoch 4/300
168/168 [==============================] - 14s 83ms/step - loss: 0.0044 - val_loss: 0.0044
Epoch 5/300
168/168 [==============================] - 16s 93ms/step - loss: 0.0044 - val_loss: 0.0044
Epoch 6/300
168/168 [==============================] - 16s 93ms/step - loss: 0.0043 - val_loss: 0.0043
Epoch 7/300
168/168 [==============================] - 14s 85ms/step - loss: 0.0043 - val_loss: 0.0043
Epoch 8/300
168/168 [==============================] - 14s 86ms/step - loss: 0.0043 - val_loss: 0.0043
Epoch 9/300
168/168 [==============================] - 14s 85ms/step - loss: 0.0042 - val_loss: 0.0043
Epoch 10/300
168/168 [==============================] - 16s 98ms/step - loss: 0.0042

In [6]:
filename = 'result/trainloss_%s.csv' % file
loss_history = np.array(history.losses_train)
np.savetxt(filename, loss_history, delimiter=",")

filename = 'result/valloss_%s.csv' % file
loss_history = np.array(history.losses_val)
np.savetxt(filename, loss_history, delimiter=",")

In [8]:
!pip install mat73

import mat73 

test_mat_file = mat73.loadmat('/content/drive/MyDrive/Datasets/test_prediction.mat')
x_test = test_mat_file["HT"]
x_test = x_test.astype(float)
input_test_samples = np.concatenate((x_test[:, 0:3], x_test[:, 4:7]), axis=1)
output_test_samples = np.concatenate((x_test[:, 3:4], x_test[:, 7:8]), axis=1)
input_test_samples = np.reshape(input_test_samples, (-1, 2*L, Nr, Nt, K))
output_test_samples = np.reshape(output_test_samples, (-1, 2, Nr, Nt, K))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
# Now is the time to test our model and store in x_hat
tStart = time.time()
x_hat = CsiNet.predict(input_test_samples)
tEnd = time.time()
print("It cost %f sec" % ((tEnd - tStart)/input_test_samples.shape[0]))

695/695 [==============================] - 4s 5ms/step
It cost 0.000308 sec


In [11]:

def NMSE(Hi, Hi_hat):
    Hi_real = np.reshape(Hi[:, 0, :, :, :], (len(Hi), -1))
    Hi_img = np.reshape(Hi[:, 1, :, :, :], (len(Hi), -1))
    Hi_complex = (Hi_real-0.5)+1j*(Hi_img-0.5)

    Hi_hat_real = np.reshape(Hi_hat[:, 0, :, :, :], (len(Hi_hat), -1))
    Hi_hat_img = np.reshape(Hi_hat[:, 1, :, :, :], (len(Hi_hat), -1))
    Hi_hat_complex = (Hi_hat_real-0.5)+1j*(Hi_hat_img-0.5)

    mse = np.sum(np.square(np.abs(Hi_complex - Hi_hat_complex)), axis=(1))
    power = np.sum(np.abs(Hi_complex) ** 2, axis=(1))
    NMSE = 10 * np.log10(np.mean(mse) / np.maximum(np.mean(power), 1e-10))
    return NMSE



# print(x_hat.shape)
# print(output_test_samples.shape)

print("NMSE is ", NMSE(output_test_samples, x_hat))
filename = "result/decoded_%s.csv" % file
x_hat1 = np.reshape(x_hat, (len(x_hat), -1))
np.savetxt(filename, x_hat1, delimiter=",")



NMSE is  -13.961168386732453


In [13]:
# Serialize CSINET model to JSON
model_json = CsiNet.to_json()
outfile = "result/model_%s.json" % file
with open(outfile, "w") as json_file:
    json_file.write(model_json)

# Serialize weights to HDF5
outfile = "result/model_%s.h5" % file
CsiNet.save_weights(outfile)

# Save model in tensorflow format
CsiNet.save("tensorflow/model_%s" % file)

In [ ]:
# first we do some data preprocessing that is extracting what is required from
# the given data and reshape it into the required input shape
# same for the output data
# and same for the test data

In [ ]:
# in this piece of code we're trying to fit the model
# we're keeping number of epochs as 300 and a batch size of 512

In [ ]:
# then we will find the normalised mean squared error
# and compare it with the actual results we got in the research paper

In [14]:
import shutil
shutil.make_archive('/content/result', 'zip', '/content/result')

'/content/result.zip'

In [15]:
from google.colab import files
files.download('/content/result.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
import shutil
shutil.make_archive('/content/tensorflow', 'zip', '/content/tensorflow')

'/content/tensorflow.zip'

In [17]:
from google.colab import files
files.download('/content/tensorflow.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>